In [ ]:
### This script will display the network in its entirety. 
### If you want you can enable physics for drawing the network
### and display the physics buttons to find a good visualization.
### If you want to visualize a local region of the network you 
### can use the third script.

from pyvis.network import Network
from pathlib import Path
import networkx as nx

folder = Path() / "data"

### By default the full network of the airlines is displayed. If you want to
### only see the network of a specific air_line, change air_line_network.gml
### in the next line by one of the following:
### Air_France.gml
### Altalia.gml
### British_Airways.gml
### Easyjet.gml
### KLM.gml
### Lufthansa.gml
### Ryanair.gml
### Scandinavian_Airlines.gml

air_line_gml_file = folder / "Air_France.gml"

### Read the gml file as a network
air_line_graph = nx.read_gml(air_line_gml_file)



### Create a pyvis network from G
air_line_network = Network("800px", width="100%", bgcolor="#222222", font_color="white", \
                        heading="EU Airline Network")
air_line_network.from_nx(air_line_graph)

### Manipulate note attributes for display purposes
for node in air_line_network.nodes:
    #We first set the size of the node to be equal to its degree/2
    node["size"] = int(air_line_graph.degree[node["label"]])/2
    
    ### We set the label of the node to the city name. Then we set the 
    ### title to be the name of the city in bold and include the ICAO and
    ### country code .
    node["label"] = node["city"]
    node["title"] = "<b>" + node["city"] + "</b> <br>" + node['ICAO'] + ", " + node["country"]  

### Comment the next line to enable the physics for drawing the graph
air_line_network.toggle_physics(False)

### You can control the physics in the output page by uncommenting the next line
### By switching of the "Physics Enabled" box the nodes will stop moving
air_line_network.show_buttons(filter_=['physics'])

air_line_network.show("air_line_network.html")

In [11]:
import random as rnd

def compute_local_neighborhood(G,node,size):
    if (node not in G.nodes):
        idx = int(rnd.uniform(0, len(G.nodes)))
        node = list(G.nodes)[idx]
    
    if (size <= 0):
        size = 2
        
    local_graph = nx.Graph()
    
    
    current_size = 0
    
    current_neighborhood = set()
    next_neighborhood = set()
    
    current_neighborhood.add(node)
    
    while current_size < size:
        current_size += 1
        for node in current_neighborhood:
            neighbors = G.adj[node]
            ICAO = G.nodes[node]['ICAO']
            city = G.nodes[node]['city']
            country = G.nodes[node]['country']
            local_graph.add_node(node, ICAO=ICAO,city=city,country=country)
            for neighbor in neighbors:
                ICAO = G.nodes[neighbor]['ICAO']
                city = G.nodes[neighbor]['city']
                country = G.nodes[neighbor]['country']
                local_graph.add_node(neighbor, ICAO=ICAO,city=city,country=country)
                local_graph.add_edge(node,neighbor)
                next_neighborhood.add(neighbor)
        
        current_neighborhood = next_neighborhood.copy()
        next_neighborhood = set()
        
    
    return local_graph

In [14]:
### This script will draw a subnetwork of the entire network. For this
### it will take a node [root_node] and look at the neighborhood up to
### size [size_of_neighborhood]. If this size is 1, it will only show
### the neighbors. For size 2 it will also show all the neighbors of the
### neighbors. And so on. If no root_node is specified a random node will
### be selected that has degree > 0.
### NOTE: do not forget to run the script above so that the function
### compute_local_neighborhood is defined.

from pyvis.network import Network
from pathlib import Path
import networkx as nx
import random as rnd

root_node = ""
size_of_neighborhood = 2

folder = Path() / "data"

air_lines_gml_file = folder / "air_line_network.gml"

### Read the gml file as a graph
air_lines_graph = nx.read_gml(air_lines_gml_file)

### If the specified root is not in the graph we select a node with
### degree > 0 at random
if (root_node not in air_lines_graph.nodes):
    idx = int(rnd.uniform(0, len(air_lines_graph.nodes)))
    root_node = list(air_lines_graph.nodes)[idx]
    while (air_lines_graph.degree[root_node] <= 0):
        idx = int(rnd.uniform(0, len(air_lines_graph.nodes)))
        root_node = list(air_lines_graph.nodes)[idx]
        
heading = f"EU Airlines Network, Local view from: {air_lines_graph.nodes[root_node]['city']}" \
            + f" of size {size_of_neighborhood}"

small_air_lines_graph = compute_local_neighborhood(air_lines_graph,root_node,size_of_neighborhood)

### Create a pyvis network from the graph
small_air_lines_network = Network("800px", width="100%", bgcolor="#222222", font_color="white", \
                           heading=heading)
small_air_lines_network.from_nx(small_air_lines_graph)

### Manipulate note attributes for display purposes
for node in small_air_lines_network.nodes:
    if (node["id"] == root_node):
        node.update({'x':0, 'y':0, 'color':'red'})
        node["size"] = 10
    else:
        node["size"] = 2

    ### We set the label of the node to the city name. Then we set the 
    ### title to be the name of the city in bold and include the ICAO and
    ### country code .
    node["label"] = node["city"]
    node["title"] = "<b>" + node["city"] + "</b> <br>" + node['ICAO'] + ", " + node["country"]  
    
### Comment the next line to enable the physics for drawing the graph
small_air_lines_network.toggle_physics(False)

### You can control the physics in the output page by uncommenting the next line
### By switching of the "Physics Enabled" box the nodes will stop moving
#small_air_lines_network.show_buttons(filter_=['physics'])

small_air_lines_network.show("eu_roads.html")